In [1]:
from pyspark import SparkContext, SparkConf, SQLContext, HiveContext, SparkFiles
from pyspark.sql import *
import math
from collections import defaultdict
from pyspark.sql.functions import udf, struct
from pyspark.sql.types import StringType, MapType, ArrayType
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, NGram, OneHotEncoder, StringIndexer, CountVectorizer
from pyspark.ml.feature import VectorIndexer, VectorAssembler
from pyspark.mllib.util import MLUtils
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

from pyspark.sql.functions import *
import os
import re
import sys

### Read our train and test data in as Pyspark Dataframes, using the inferSchema option

In [2]:
PATH = "./"
train_df = (spark.read
    .option("header", "true")
    .option("mode", "DROPMALFORMED")
    .option("inferSchema", "true")
    .csv('{}train.tsv'.format(PATH), sep="\t"))

test_df = (spark.read
    .option("header", "true")
    .option("mode", "DROPMALFORMED")
    .option("inferSchema", "true")
    .csv('{}test.tsv'.format(PATH), sep="\t"))

### Basic NLP Feature Engineering

In [3]:
tokenizer = RegexTokenizer(inputCol="item_description", outputCol="tokens", pattern="\\W")
train_df_temp = tokenizer.transform(train_df)
remover = StopWordsRemover(inputCol="tokens", outputCol="unigrams")
train_df_temp = remover.transform(train_df_temp)
ngram = NGram(n=2, inputCol="unigrams", outputCol="bigrams")
train_df_temp = ngram.transform(train_df_temp)
def replace_blank(x):
    return when((col(x) == "") | col(x).isNull(), "None").otherwise(col(x))
train_df_temp = train_df_temp.withColumn('brand_name', replace_blank('brand_name'))

tokenizer = RegexTokenizer(inputCol="item_description", outputCol="tokens", pattern="\\W")
test_df_temp = tokenizer.transform(test_df)
remover = StopWordsRemover(inputCol="tokens", outputCol="unigrams")
test_df_temp = remover.transform(test_df_temp)
ngram = NGram(n=2, inputCol="unigrams", outputCol="bigrams")
test_df_temp = ngram.transform(test_df_temp)
def replace_blank(x):
    return when((col(x) == "") | col(x).isNull(), "None").otherwise(col(x))
test_df_temp = train_df_temp.withColumn('brand_name', replace_blank('brand_name'))

### One hot encoding of unigrams and bigrams
Let's one hot encode the ungigrams an bigrams to make it easier to select and experiment with features when we move to sklearn models

In [4]:
stringIndexer = StringIndexer(inputCol="brand_name", outputCol="brand_category_index")
model = stringIndexer.fit(train_df_temp)
train_df_temp = model.transform(train_df_temp)
encoder = OneHotEncoder(inputCol="brand_category_index", outputCol="categoryVec")
train_df_temp = encoder.transform(train_df_temp)

stringIndexer = StringIndexer(inputCol="brand_name", outputCol="brand_category_index")
model = stringIndexer.fit(test_df_temp)
test_df_temp = model.transform(test_df_temp)
encoder = OneHotEncoder(inputCol="brand_category_index", outputCol="categoryVec")
test_df_temp = encoder.transform(test_df_temp)

In [6]:
cv = CountVectorizer(inputCol="unigrams", outputCol="vectors")
model = cv.fit(train_df_temp)
train_df_temp = model.transform(train_df_temp)

model = cv.fit(test_df_temp)
test_df_temp = model.transform(test_df_temp)

Py4JJavaError: An error occurred while calling o260.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 10.0 failed 1 times, most recent failure: Lost task 3.0 in stage 10.0 (TID 33, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$createTransformFunc$2: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCodegenExec.scala:395)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:408)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:439)
	at org.apache.spark.util.collecti

### Join the unigrams and bigrams fields to write out to csv format

In [ ]:
train_df_temp = train_df_temp.withColumn('unigrams_string', concat_ws(',', 'unigrams'))
train_df_temp = train_df_temp.withColumn('bigrams_string', concat_ws(',', 'bigrams'))

In [ ]:
ignore = ['train_id', 'test_id', 'price']
assembler = VectorAssembler(
    inputCols=[x for x in df.columns if x not in ignore],
    outputCol='features')
assembler.transform(train_df_temp)

In [ ]:
# train_features_df = train_df_temp.select('train_id', 'name', 'item_condition_id', 'category_name', 'brand_name', 'price', 'shipping', 'item_description', 'unigrams_string', 'bigrams_string')
# test_features_df = train_df_temp.select('test_id', 'name', 'item_condition_id', 'category_name', 'brand_name', 'shipping', 'item_description', 'unigrams_string', 'bigrams_string')
# train_features_df.toPandas().to_csv('/tmp/train_features.csv')
# test_features_df.toPandas().to_csv('/tmp/test_features.csv')


TODO: combine these steps in a pipeline